Input File Creation
===========
First let's start with some tools to create input files for a given deployment schedule.

In [1]:
import os
import uuid
import json
import subprocess
from math import ceil
from copy import deepcopy

import numpy as np
import pandas as pd
import cymetric as cym
%matplotlib inline
import matplotlib.pyplot as plt

import dtw

/home/scopatz/miniconda/lib/python3.5/importlib/_bootstrap.py:222: QAWarning: pyne.data is not yet QA compliant.
  return f(*args, **kwds)
/home/scopatz/miniconda/lib/python3.5/importlib/_bootstrap.py:222: QAWarning: pyne.material is not yet QA compliant.
  return f(*args, **kwds)
/home/scopatz/miniconda/lib/python3.5/importlib/_bootstrap.py:222: QAWarning: pyne.enrichment is not yet QA compliant.
  return f(*args, **kwds)


In [2]:
with open('once-through.json') as f:
    BASE_SIM = json.load(f)
DURATION = BASE_SIM['simulation']['control']['duration']
YEARS = ceil(DURATION / 12)
MONTH_SHUFFLE = (0, 6, 9, 3, 7, 5, 11, 1, 4, 8, 10, 2)
NULL_SCHEDULE = {'build_times': [{'val': 1}], 
                 'n_build': [{'val': 0}], 
                 'prototypes': [{'val': 'LWR'}]}
LWR_PROTOTYPE = {'val': 'LWR'}
OPT_H5 = 'opt.h5'

In [3]:
BASE_SIM['simulation']['region']['institution']['config']['DeployInst']

{'build_times': [{'val': 1}],
 'n_build': [{'val': 0}],
 'prototypes': [{'val': 'LWR'}]}

In [50]:
def deploy_inst_schedule(Θ):
    if np.sum(Θ) == 0: 
        return NULL_SCHEDULE
    sched = {'build_times': {'val': []},
             'n_build': {'val': []},
             'prototypes': {'val': []}}
    build_times = sched['build_times']['val']
    n_build = sched['n_build']['val']
    prototypes = sched['prototypes']['val']
    m = 0
    for i, θ in enumerate(Θ):
        if θ <= 0:
            continue
        build_times.append(i*12 + MONTH_SHUFFLE[m])
        n_build.append(int(θ))
        prototypes.append('LWR')
        m = (m + 1) % 12
    return sched

def make_sim(Θ, fname='sim.json'):
    sim = deepcopy(BASE_SIM)
    inst = sim['simulation']['region']['institution']
    inst['config']['DeployInst'] = deploy_inst_schedule(Θ)
    with open(fname, 'w') as f:
        json.dump(sim, f)
    return sim

In [5]:
s = make_sim([])

In [6]:
s['simulation']['region']['institution']['config']['DeployInst']

{'build_times': [{'val': 1}],
 'n_build': [{'val': 0}],
 'prototypes': [{'val': 'LWR'}]}

Simulate
=========
Now let's build some tools to run simulations and extract a GWe time series.

In [7]:
def run(fname='sim.json', out=OPT_H5):
    """Runs a simulation and returns the sim id."""
    cmd = ['cyclus', '-o', out, fname]
    proc = subprocess.run(cmd, check=True, universal_newlines=True, stdout=subprocess.PIPE)
    simid = proc.stdout.rsplit(None, 1)[-1]
    return simid

ZERO_GWE = pd.DataFrame({'GWe': np.zeros(YEARS)}, index=np.arange(YEARS))
ZERO_GWE.index.name = 'Time'

def extract_gwe(simid, out=OPT_H5):
    """Computes the annual GWe for a simulation."""
    db = cym.dbopen(out)
    evaler = cym.Evaluator(db)
    raw = evaler.eval('TimeSeriesPower', conds=[('SimId', '==', uuid.UUID(simid))])
    ano = pd.DataFrame({'Time': raw.Time.apply(lambda x: x//12), 
                        'GWe': raw.Value.apply(lambda x: 1e-3*x/12)})
    gwe = ano.groupby('Time').sum()
    gwe = (gwe + ZERO_GWE).fillna(0.0)
    return np.array(gwe.GWe)

Distancing
========
Now let's build some tools to distance between a GWe time series and a demand curve.

In [8]:
DEFAULT_DEMAND = 90 * (1.01**np.arange(YEARS))  # 1% growth

In [20]:
def d(g, f=None):
    """The dynamic time warping distance between a GWe time series and a demand curve."""
    f = DEFAULT_DEMAND if f is None else f
    rtn = dtw.distance(f[:, np.newaxis], g[:, np.newaxis])
    return rtn

def gwed(Θ, f=None):
    """For a given deployment schedule Θ, return the GWe time series and the distance 
    to the demand function f.
    """
    make_sim(Θ)
    simid = run()
    gwe = extract_gwe(simid)
    dΘ = d(gwe, f=f)
    return gwe, dΘ

Initialize Optimization
===============
Now let's start with a couple of simple simulations

In [44]:
N = np.asarray(np.ceil(4*(1.01)**np.arange(YEARS)), dtype=int)  # max annual deployments
Θs = [] # deployment schedules
G = []  # GWe per sim
D = []  # distances per sim
if os.path.isfile(OPT_H5):
    os.remove(OPT_H5)

In [33]:
def add_sim(Θ, f=None):
    """Add a simulation to the known simulations by performing the simulation."""
    g_s, d_s = gwed(Θ, f=f)
    Θs.append(Θ)
    G.append(g_s)
    D.append(d_s)

First, add a schedule where nothing is deployed, leaving the initial facilities to retire.

In [45]:
add_sim(np.zeros(YEARS, dtype=int))

Next, add a simulation that is the max deployment schedule to bound the space

In [51]:
add_sim(N)

CalledProcessError: Command '['cyclus', '-o', 'opt.h5', 'sim.json']' returned non-zero exit status 1

In [52]:
deploy_inst_schedule(N)

{'build_times': {'val': [0,
   18,
   33,
   39,
   55,
   65,
   83,
   85,
   100,
   116,
   130,
   134,
   144,
   162,
   177,
   183,
   199,
   209,
   227,
   229,
   244,
   260,
   274,
   278,
   288,
   306,
   321,
   327,
   343,
   353,
   371,
   373,
   388,
   404,
   418,
   422,
   432,
   450,
   465,
   471,
   487,
   497,
   515,
   517,
   532,
   548,
   562,
   566,
   576,
   594]},
 'n_build': {'val': [4,
   5,
   5,
   5,
   5,
   5,
   5,
   5,
   5,
   5,
   5,
   5,
   5,
   5,
   5,
   5,
   5,
   5,
   5,
   5,
   5,
   5,
   5,
   6,
   6,
   6,
   6,
   6,
   6,
   6,
   6,
   6,
   6,
   6,
   6,
   6,
   6,
   6,
   6,
   6,
   6,
   7,
   7,
   7,
   7,
   7,
   7,
   7,
   7,
   7]},
 'prototypes': {'val': ['LWR',
   'LWR',
   'LWR',
   'LWR',
   'LWR',
   'LWR',
   'LWR',
   'LWR',
   'LWR',
   'LWR',
   'LWR',
   'LWR',
   'LWR',
   'LWR',
   'LWR',
   'LWR',
   'LWR',
   'LWR',
   'LWR',
   'LWR',
   'LWR',
   'LWR',
   'LWR',
   'LWR',
   '

In [53]:
!cyclus --json-to-xml sim.json

<simulation>
  <archetypes>
    <spec>
      <lib>cycamore</lib>
      <name>DeployInst</name>
    </spec>
    <spec>
      <lib>cycamore</lib>
      <name>Enrichment</name>
    </spec>
    <spec>
      <lib>cycamore</lib>
      <name>Reactor</name>
    </spec>
    <spec>
      <lib>cycamore</lib>
      <name>Source</name>
    </spec>
    <spec>
      <lib>cycamore</lib>
      <name>Sink</name>
    </spec>
    <spec>
      <lib>agents</lib>
      <name>NullRegion</name>
    </spec>
  </archetypes>
  <commodity>
    <name>U-ore</name>
    <solution_priority>1</solution_priority>
  </commodity>
  <commodity>
    <name>Fresh-UOX-Fuel</name>
    <solution_priority>1</solution_priority>
  </commodity>
  <commodity>
    <name>Enrich-Tails</name>
    <solution_priority>1</solution_priority>
  </commodity>
  <commodity>
    <name>Used-UOX-Fuel</name>
    <solution_priority>1</solution_priority>
  </commodity>
  <control>
    <duration>600</duration>
    <startmonth>1</startmonth>
    <startyea

In [17]:
Θ = 5

In [18]:
print(Θ)        

5
